In [1]:
import threading
import time
from pynq.lib import Pmod_IO
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

In [2]:
#create Pmod_IO objects for LED pins
gpio_pin_1 = Pmod_IO(base.PMODB,1,'out')
gpio_pin_2 = Pmod_IO(base.PMODB,2,'out')
gpio_pin_3 = Pmod_IO(base.PMODB,3,'out')

#store thses pins in a dictionary to easily refrence based on color
pins = {
    'red'   : gpio_pin_3, 
    'green' : gpio_pin_2,
    'blue'  : gpio_pin_1
} 

def blink(t, d, n):
    '''
    Function to blink the LEDs
    Params:
      t: number of times to blink the LED
      d: duration (in seconds) for the LED to be on/off
      n: index of the LED (0 to 4) - 4 is off board LED
    '''
    if n == 4:
        for i in range(1,t+1):
            led_pwr = i % 2
            pins['green'].write(led_pwr)
            time.sleep(d)
        pins['green'].write(0)
    else:
        for i in range(t):
            base.leds[n].toggle()
            time.sleep(d)
        base.leds[n].off()
    
def turn_leds_off():
    
    for color in pins:
        pins[color].write(0)
    for n in range(4):
        base.leds[n].off()

In [3]:
turn_leds_off()

In [3]:
blink(50,0.02,3)

In [15]:
def philosopher_t(f0, f1, f2, f3, f4, rtt, num):
    '''
    Worker function to try and acquire resource to eat
    f0-f4 - forks (locks)
    num: index representing the LED and thread number.
    '''
    
    '''
    using_fork0 = False
    using_fork1 = False
    using_fork2 = False
    using_fork3 = False
    using_fork4 = False
    '''
    
    if num == 0:
        left_fork = f0
        right_fork = f4
    elif num == 1:
        left_fork = f1
        right_fork = f0
    elif num == 2:
        left_fork = f2
        right_fork = f1
    elif num == 3:
        left_fork = f3
        right_fork = f2
    elif num == 4:
        left_fork = f4
        right_fork = f3
        
    using_left_fork = False
    using_right_fork = False
    has_rtt = False
    
    #only want to take ANY forks if you can get both
    #try non-blocking acquire for one fork, then release if timeout for getting the other???
    
    #TO DO: add button interrupt to stop
        
    for i in range(4):
        if using_left_fork:
            left_fork.release()
        if using_right_fork:
            right_fork.release()
        
        #Entering Starving state
        print("{}: Philosopher {} is starving".format(time.strftime("%H:%M:%S"),num))
        while not (using_left_fork and using_right_fork):
            has_rtt = rtt.acquire(True)
            #print("Philosopher {} has rtt".format(num))
            using_left_fork = left_fork.acquire(False)
            if using_left_fork:
                #print("Philosopher {} has left fork".format(num))
                using_right_fork = right_fork.acquire(False)
                if not using_right_fork:
                    left_fork.release()
                    rtt.release()
                    using_left_fork = False
                    has_rtt = False
                    #print("Philosopher {} put down left fork and relinquished rtt".format(num))
                    time.sleep(1)
            else:
                rtt.release()
                has_rtt = False
                #print("Philosopher {} relinquished rtt".format(num))
                time.sleep(1)

        #has both forks, entering Eating state
        rtt.release()
        has_rtt = False
        print("{}: Philosopher {} is eating".format(time.strftime("%H:%M:%S"),num))
        blink(250, 0.02, num)
        
        #done eating
        left_fork.release()
        right_fork.release()
        using_left_fork = False
        using_right_fork = False
        print("{}: Philosopher {} is napping".format(time.strftime("%H:%M:%S"),num))
        blink(10, 1, num)
        
    print("Worker {} is done.".format(num))

In [16]:
# Initialize and launch the threads
threads = []
fork0 = threading.Lock()
fork1 = threading.Lock()
fork2 = threading.Lock()
fork3 = threading.Lock()
fork4 = threading.Lock()
rtt = threading.Lock() #right to take
for i in range(5):
    t = threading.Thread(target=philosopher_t, args=(fork0, fork1, fork2, fork3, fork4, rtt, i))
    threads.append(t)
    t.start()

for t in threads:
    name = t.getName()
    t.join()
    print('{} joined'.format(name))

19:21:06: Philosopher 0 is starving
19:21:06: Philosopher 0 is eating
19:21:06: Philosopher 1 is starving
19:21:06: Philosopher 2 is starving
19:21:06: Philosopher 2 is eating
19:21:06: Philosopher 3 is starving
19:21:06: Philosopher 4 is starving
19:21:11: Philosopher 0 is napping
19:21:11: Philosopher 2 is napping
19:21:12: Philosopher 1 is eating
19:21:12: Philosopher 3 is eating
19:21:17: Philosopher 1 is napping
19:21:17: Philosopher 3 is napping
19:21:18: Philosopher 4 is eating
19:21:21: Philosopher 0 is starving
19:21:21: Philosopher 2 is starving
19:21:21: Philosopher 2 is eating
19:21:23: Philosopher 4 is napping
19:21:24: Philosopher 0 is eating
19:21:26: Philosopher 2 is napping
19:21:27: Philosopher 1 is starving
19:21:27: Philosopher 3 is starving
19:21:27: Philosopher 3 is eating
19:21:29: Philosopher 0 is napping
19:21:30: Philosopher 1 is eating
19:21:32: Philosopher 3 is napping
19:21:33: Philosopher 4 is starving
19:21:33: Philosopher 4 is eating
19:21:35: Philosophe